In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import fast_dsift, no_op, dsift, sparse_hog
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images

from menpofit.aam import (
    AAMBuilder, PatchAAMBuilder, LinearAAMBuilder, LinearPatchAAMBuilder, PartsAAMBuilder,
    SupervisedDescentAAMFitter,
    ProjectOutNewton, ProjectOutGaussNewton)

from menpofit.visualize import visualize_aam, visualize_fitting_result

## Load training data

In [ ]:
training_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/',  
                           verbose=True, max_images=None):
    print i
    i.rescale_landmarks_to_diagonal_range(200)
    i = i.crop_to_landmarks_proportion(0.5)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    training_images.append(i)

In [ ]:
visualize_images(training_images)

## Load test data

In [ ]:
test_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/testset/', 
                           verbose=True, max_images=None):    
    i.rescale_landmarks_to_diagonal_range(200)
    i.crop_to_landmarks_proportion_inplace(0.5)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    test_images.append(i)

In [ ]:
visualize_images(test_images)

## Build Active Appearance Models

In [ ]:
features = fast_dsift
diagonal = 200
scales = (1, .5)
scale_shapes = False
scale_features = True
max_shape_components= 25
max_appearance_components = None

patch_shape = (17, 17)

normalize_parts = no_op

#### AAM

In [ ]:
# aam_builder = AAMBuilder(
#     features=features,
#     diagonal=diagonal, 
#     scales=scales, 
#     scale_shapes=scale_shapes,
#     scale_features=scale_features,
#     max_shape_components=max_shape_components,
#     max_appearance_components=max_appearance_components)

# aam = aam_builder.build(
#     training_images, 
#     group='ibug_face_66', 
#     verbose=True)

#### Patch based AAM

In [ ]:
# patch_aam_builder = PatchAAMBuilder(
#     patch_shape=patch_shape,
#     features=features,
#     diagonal=diagonal, 
#     scales=scales, 
#     scale_shapes=scale_shapes,
#     scale_features=scale_features,
#     max_shape_components=max_shape_components,
#     max_appearance_components=max_appearance_components)

# patch_aam = patch_aam_builder.build(
#     training_images, 
#     group='ibug_face_66', 
#     verbose=True)

#### Linear AAM

In [ ]:
# linear_aam_builder = LinearAAMBuilder(
#     features=features,
#     diagonal=diagonal, 
#     scales=scales, 
#     scale_shapes=scale_shapes,
#     scale_features=scale_features,
#     max_shape_components=max_shape_components,
#     max_appearance_components=max_appearance_components)

# linear_aam = linear_aam_builder.build(
#     training_images, 
#     group='ibug_face_66', 
#     verbose=True)

#### Linear Patch based AAM

In [ ]:
# linear_patch_aam_builder = LinearPatchAAMBuilder(
#     patch_shape=patch_shape,
#     features=features,
#     diagonal=diagonal, 
#     scales=scales, 
#     scale_shapes=scale_shapes,
#     scale_features=scale_features,
#     max_shape_components=max_shape_components,
#     max_appearance_components=max_appearance_components)

# linear_patch_aam = linear_patch_aam_builder.build(
#     training_images, 
#     group='ibug_face_66', 
#     verbose=True)

#### Parts based AAM

In [ ]:
parts_aam_builder = PartsAAMBuilder(
    patch_shape=patch_shape, 
    features=features,
    normalize_parts=normalize_parts,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components,
    max_appearance_components=max_appearance_components)

parts_aam = parts_aam_builder.build(
    training_images, 
    group='ibug_face_66', 
    verbose=True)

## Fit Active Appearance Models

In [ ]:
sd_algorithm_cls = ProjectOutGaussNewton
n_shape = [5, 15]
n_appearance = [50, 100]
n_perturbations = 10
noise_std = 0.05
sampling_step = [8, 8]

max_iters = [5, 3]
gamma = 1000000
psi = 1000

In [ ]:
sampling_mask = parts_mask = np.require(np.zeros(patch_shape, dtype=np.bool))
sampling_mask[2::4, 2::4] = True
imshow(sampling_mask)

sampling_masks = [sampling_mask]

In [ ]:
sampling_mask = parts_mask = np.require(np.zeros(patch_shape, dtype=np.bool))
sampling_mask[5:12:2, 5:12:2] = True
imshow(sampling_mask)
sampling_masks.append(sampling_mask)

#### AAM Fitter

In [ ]:
# aam_fitter = CRAAMFitter(
#     aam, 
#     cr_algorithm_cls=cr_algorithm_cls, 
#     n_shape=n_shape, 
#     n_appearance=n_appearance,
#     n_perturbations=n_perturbations,
#     max_iters=max_iters,
#     sampling=sampling_step)

# aam_fitter.train(
#     training_images, 
#     group='ibug_face_66', 
#     gamma=gamma,
#     verbose=True)

#### Patch based AAM Fitter

In [ ]:
# patch_aam_fitter = CRAAMFitter(
#     patch_aam, 
#     cr_algorithm_cls=cr_algorithm_cls, 
#     n_shape=n_shape, 
#     n_appearance=n_appearance,
#     n_perturbations=n_perturbations,
#     max_iters=max_iters,
#     sampling=sampling_step)

# patch_aam_fitter.train(
#     training_images, 
#     group='ibug_face_66', 
#     gamma=gamma,
#     verbose=True)

#### Linear AAM Fitter

In [ ]:
# linear_aam_fitter = CRAAMFitter(
#     linear_aam, 
#     cr_algorithm_cls=cr_algorithm_cls, 
#     n_shape=n_shape, 
#     n_appearance=n_appearance,
#     n_perturbations=n_perturbations,
#     max_iters=max_iters,
#     sampling=sampling_step)

# linear_aam_fitter.train(
#     training_images, 
#     group='ibug_face_66', 
#     gamma=gamma,
#     verbose=True)

#### Linear Patch based AAM Fitter

In [ ]:
# linear_patch_aam_fitter = CRAAMFitter(
#     linear_patch_aam, 
#     cr_algorithm_cls=cr_algorithm_cls, 
#     n_shape=n_shape, 
#     n_appearance=n_appearance,
#     n_perturbations=n_perturbations,
#     max_iters=max_iters,
#     sampling=sampling_step)

# linear_patch_aam_fitter.train(
#     training_images, 
#     group='ibug_face_66', 
#     gamma=gamma,
#     verbose=True)

#### Parts based AAM Fitter

In [ ]:
parts_aam_fitter = SupervisedDescentAAMFitter(
    parts_aam, 
    sd_algorithm_cls=sd_algorithm_cls, 
    n_shape=n_shape, 
    n_appearance=n_appearance,
    n_perturbations=n_perturbations,
    noise_std=noise_std,
    max_iters=max_iters,
    sampling=sampling_masks)

parts_aam_fitter.train(
    training_images, 
    group='ibug_face_66', 
    gamma=gamma,
    #psi=psi,
    dtype=np.float32,
    verbose=True)

#### Fitting loop

In [ ]:
np.random.seed(seed=0)

fitters = [#aam_fitter,
           #patch_aam_fitter,
           #linear_aam_fitter,
           #linear_patch_aam_fitter,
           parts_aam_fitter]

fitters_results =  [[]]

for j, i in enumerate(test_images):
    
    gt_s = i.landmarks['ibug_face_66'].lms
    s = fitters[0].noisy_shape_from_shape(gt_s, noise_std=0.05)

    for k, fitter in enumerate(fitters):
        
        fr = fitter.fit(i, s, gt_shape=gt_s)
        fr.downscale = 0.5   
        fitters_results[k].append(fr)
    
        print 'Image: ', j
        print fr

## Visualize Results

In [ ]:
visualize_fitting_result(fitters_results[0])